In [7]:
pip install toolhouse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 7.6 MB/s eta 0:00:00


In [8]:
pip install groq

In [28]:
# Import packages
from toolhouse import Toolhouse
from groq import Groq

In [29]:
client = Groq(api_key="groq api key")
th = Toolhouse(api_key="tool house api key")

In [30]:
MODEL = "llama-3.3-70b-versatile"

In [31]:
print('TOOLS AVAILABLE:')
for tool in th.get_tools():
    print(f"Name: {tool['function']['name']}")
    print(f"Type: {tool['type']}")
    print(f"Description: {tool['function']['description']}")
    print('Required Parameters:')
    for required_parameter in tool['required']:
      print(f"{required_parameter} - {tool['function']['parameters']['properties'][required_parameter]['description']}")
    print('\n')

TOOLS AVAILABLE:
Name: exa_web_search
Type: function
Description: Performs a web search using a search engine like Google Search. You can use the Google search query syntax. By default, web search returns up to 5 results, but your LLM can specify a different number of results. To do so, you can prompt your LLM to pass a 'num_results' argument to your query.
Required Parameters:
query - The search query. You can use natural language to describe what you want to search for.


Name: pdf2csv
Type: function
Description: Allows you to get the contents of a table from a PDF file provided by the user. Use this tool any time you need to accurately extract data from a PDF file. You can also use this tool whenever the user asks you to do so. IMPORTANT: do not create the URL. Ensure the user gave you a valid URL before calling this tool.
Required Parameters:
pdf_url - The url of the PDF you want to parse.


Name: random_emoji
Type: function
Description: Selects a random emoji (and sometimes a rand

In [32]:
# User message to the LLM
messages = [
  {
    "role": "user",
    "content": "Generate and run code to solve this equation: x = 409830948 / 9834294824.",
  }
]

In [33]:
# Groq API response with Toolhouse tools
response = client.chat.completions.create(
  model=MODEL,
  messages=messages,
  # Passes Code Execution as a tool
  tools=th.get_tools(),
)

In [40]:
tools_called = response.choices[0].message.tool_calls
print(tools_called)

None


In [41]:
tool_run = th.run_tools(response)
messages.extend(tool_run)


for message in tool_run:
    print(f"Role: {message['role']}")
    if 'tool_calls' in message:
        print(f"Tool Calls: {message['tool_calls']}")
    if 'tool_call_id' in message:
        print(f"Tool Call ID: {message['tool_call_id']}")
    print(f"Content: {message['content']}")
    print('\n')

In [24]:
import os
from toolhouse import Toolhouse
# 👋 Make sure you've also installed the Groq SDK through: pip install groq
from groq import Groq

client = Groq(api_key="groq api key")
MODEL = "llama-3.3-70b-versatile"

# If you don't specify an API key, Toolhouse will expect you
# specify one in the TOOLHOUSE_API_KEY env variable.
th = Toolhouse(api_key='TOOLHOUSE_API_KEY')
messages = [{
    "role": "user",
    "content":
        "Generate FizzBuzz code."
        "Execute it to show me the results up to 10."
}]

response = client.chat.completions.create(
  model=MODEL,
  messages=messages,
  # Passes Code Execution as a tool
  tools=th.get_tools(),
)

# Runs the Code Execution tool, gets the result,
# and appends it to the context
tool_run = th.run_tools(response)
messages.extend(tool_run)

response = client.chat.completions.create(
  model=MODEL,
  messages=messages,
  tools=th.get_tools(),
)

print(response.choices[0].message.content)

<function/code_interpreter{"code_str": "for i in range(1, 11):\n    if i % 3 == 0 and i % 5 == 0:\n        print(\"FizzBuzz\")\n    elif i % 3 == 0:\n        print(\"Fizz\")\n    elif i % 5 == 0:\n        print(\"Buzz\")\n    else:\n        print(i)"}</function>
